# Reproject
By Cascade Tuholske Aug 2021  

Notebook reprojects reclassified GHS-SMOD (urban/rural only) to WGS-84 from mollweide. 

In [1]:
# dependencies
import os
import rioxarray as rio
import numpy as np

In [2]:
# paths
gpw_fn = os.path.join('/Users/cpt2136/Github/PopGridCompare/data/raw/gpw_v4/gpw-v4-population-count-rev11_2015_30_sec_tif/gpw_v4_population_count_rev11_2015_30_sec.tif')
gpw = rio.open_rasterio(gpw_fn)

smod_fn = os.path.join('../../PopGridCompare/data/raw/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif')
smod = rio.open_rasterio(smod_fn)

### Reclassify urban, rural, and peri urban based on GHS-SMOD
See for details
Florczyk A.J., Corbane C., Ehrlich D., Freire S., Kemper T., Maffenini L., Melchiorri M., Pesaresi M., Politis P., Schiavina M., Sabo F., Zanchetta L., GHSL Data Package 2019, EUR 29788 EN, Publications Office of the European Union, Luxembourg, 2019, ISBN 978-92-76-13186-1, [doi:10.2760/290498](doi:10.2760/290498), JRC 117104

In [3]:
np.unique(smod.data)

array([-200,   10,   11,   12,   13,   21,   22,   23,   30], dtype=int16)

In [4]:
# urban is 22, 23, 30, peri urban is 21, 13-11 rural, 10 water, -200 nan

# urban 333
smod.data[smod.data == 30] = 333
smod.data[smod.data == 23] = 333

# peri urban or semi dense urban 222
smod.data[smod.data == 22] = 222
smod.data[smod.data == 21] = 222

# rural 111
smod.data[smod.data == 13] = 111
smod.data[smod.data == 12] = 111
smod.data[smod.data == 11] = 111

# water/nan 0
smod.data[smod.data == 10] = 0
smod.data[smod.data == -200] = 0

In [5]:
# check 
print(np.unique(smod.data))
print('urban', np.count_nonzero(smod.data == 333))
print('peri urban',np.count_nonzero(smod.data == 222))
print('rural',np.count_nonzero(smod.data == 111))

[  0 111 222 333]
urban 997199
peri urban 978415
rural 143689939


### Re-Project

In [6]:
# reproject
smod_match = smod.rio.reproject_match(gpw)

In [7]:
# save it out 
fn_out = os.path.join('../data/interim/GHS-SMOD15-Urban-Peri-Rural-Matched.tif')
smod_match.rio.to_raster(fn_out) #named different for easy of use later
